In [1]:
# Implementation of code example from
# https://forum.orekit.org/t/python-wrapper-batchlsestimator-not-initializing/728/7


In [2]:
%matplotlib inline

from math import radians, pi
import pandas as pd
import numpy as np
import plotly.express as px


import orekit
vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir, absolutedate_to_datetime
setup_orekit_curdir()


from org.orekit.orbits import KeplerianOrbit, PositionAngle
from org.orekit.propagation.analytical import KeplerianPropagator
from org.orekit.time import AbsoluteDate, TimeScalesFactory, TimeScale
from org.orekit.utils import Constants
from org.orekit.frames import FramesFactory, TopocentricFrame
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.attitudes import LofOffset
from org.orekit.bodies import CelestialBodyFactory, OneAxisEllipsoid
from org.orekit.forces.drag import IsotropicDrag
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.maneuvers import ImpulseManeuver
from org.orekit.frames import FramesFactory, LOFType, EOPHistory
from org.orekit.models.earth.atmosphere import HarrisPriester
from org.orekit.orbits import CircularOrbit, OrbitType, PositionAngle
from org.orekit.propagation import PropagationType, SpacecraftState
from org.orekit.propagation.events import DateDetector, EventEnablingPredicateFilter, PythonEnablingPredicate,\
     PositionAngleDetector, PythonEventDetector, AbstractDetector, EventDetector
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder, DSSTPropagatorBuilder
from org.orekit.propagation.semianalytical.dsst.forces import DSSTAtmosphericDrag, DSSTNewtonianAttraction, DSSTZonal
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from orekit.pyhelpers import absolutedate_to_datetime
from org.hipparchus.linear import RealMatrix
from org.hipparchus.linear import QRDecomposer, AbstractRealMatrix
from org.hipparchus.optim.nonlinear.vector.leastsquares import GaussNewtonOptimizer
from org.orekit.estimation.leastsquares import BatchLSEstimator
from org.orekit.propagation.numerical import NumericalPropagator
from org.orekit.propagation.conversion import AbstractPropagatorBuilder

from org.orekit.estimation.measurements import AngularAzEl
from org.orekit.estimation.measurements import ObservableSatellite
from org.orekit.estimation.measurements import GroundStation
from org.orekit.estimation.sequential import KalmanEstimatorBuilder
from org.orekit.estimation.sequential import KalmanEstimator
from org.orekit.estimation.measurements import *

from orekit.pyhelpers import absolutedate_to_datetime
from collections import namedtuple
from org.orekit.time import Month
from orekit import JArray

from org.orekit.estimation.sequential import KalmanEstimator, KalmanEstimatorBuilder, CovarianceMatrixProvider, ConstantProcessNoise
from org.orekit.orbits import Orbit

from org.orekit.models.earth import ReferenceEllipsoid
from org.orekit.propagation.conversion import NumericalPropagatorBuilder
from org.orekit.attitudes import NadirPointing
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel
from org.orekit.forces.gravity import ThirdBodyAttraction
#from org.orekit.forces.drag import MarshallSolarActivityFutureEstimation
#from org.orekit.forces.drag import atmosphere


In [3]:
########## COORDINATE FRAMES, CELESTIAL BODIES AND TIME

#Ground Station Position
longitude = 21.038
latitude  = 67.8790708
altitude  = 527.0

#Defining ECI AND ECEF Coordinate Frames
gcrf = FramesFactory.getGCRF()
itrf = FramesFactory.getITRF(IERSConventions.IERS_2010, False)
eci = gcrf
ecef = itrf

#Defining Topocentric Coordinate Frame
tod = FramesFactory.getTOD(IERSConventions.IERS_2010, False)
earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                     Constants.WGS84_EARTH_FLATTENING, 
                     itrf)
station = GeodeticPoint(radians(latitude), 
                    radians(longitude), 
                    altitude)
stationFrame = TopocentricFrame(earth, station, "Esrange")

#Defining Celestial Bodies
wgs84Ellipsoid = ReferenceEllipsoid.getWgs84(ecef)
moon = CelestialBodyFactory.getMoon()
sun = CelestialBodyFactory.getSun()

#Defining Time System
utc = TimeScalesFactory.getUTC() 

In [4]:
# Given by ADS
a = 7000.0 * 1000  
e = 0.0016 
i = radians(98.0)     
omega = radians(0.2)   # perigee argument
raan = radians(90.0)  # right ascension of ascending node
lv = radians(0.1)    # True anomaly

mass = 625.0

epochDate = AbsoluteDate(2020, 1, 1, 0, 0, 00.000, utc)
initialDate = epochDate

## Inertial frame where the satellite is defined
inertialFrame = FramesFactory.getEME2000()
earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                         Constants.WGS84_EARTH_FLATTENING, 
                         inertialFrame)

## Orbit construction as Keplerian
initialOrbit = KeplerianOrbit(a, e, i, omega, raan, lv,
                              PositionAngle.TRUE,
                              inertialFrame, epochDate, Constants.WGS84_EARTH_MU)


file = pd.read_csv("export_visible_NoPert.csv")
az = file['azimuth']
el = file['elevation']
datetime = file['pv']
idx = datetime[0].find(',')


In [5]:
########## INITIAL ORBIT DETERMINATION


In [6]:
########## PROPAGATOR SETUP

#Orbit Propagator Parameters
prop_min_step = 0.001 
prop_max_step = 300.0 
prop_position_error = 10.0 

#Estimator Parameters
estimator_position_scale = 1.0 
estimator_convergence_thres = 1e-3
estimator_max_iterations = 5
estimator_max_evaluations = 35

#Measurement Parameters
azError = 0.001
elError = 0.001
azBaseWeight = 1.0
elBaseWeight = 1.0

#Defining Integrator
integratorBuilder = DormandPrince853IntegratorBuilder(prop_min_step, 
prop_max_step, prop_position_error)
propagatorBuilder = NumericalPropagatorBuilder(initialOrbit,
                                           integratorBuilder, PositionAngle.MEAN, 
estimator_position_scale)
nadirPointing = NadirPointing(eci, wgs84Ellipsoid)
propagatorBuilder.setMass(400.0)
propagatorBuilder.setAttitudeProvider(nadirPointing)

In [11]:
########## ADDING PERTURBATIONS

#Earth Gravity Field
gravityProvider = GravityFieldFactory.getConstantNormalizedProvider(64, 64)
gravityAttractionModel = HolmesFeatherstoneAttractionModel(ecef, gravityProvider)
propagatorBuilder.addForceModel(gravityAttractionModel)

#3rd Body
moon_3dbodyattraction = ThirdBodyAttraction(moon)
#propagatorBuilder.addForceModel(moon_3dbodyattraction)
sun_3dbodyattraction = ThirdBodyAttraction(sun)
#propagatorBuilder.addForceModel(sun_3dbodyattraction)


#Atmospheric Drag
#msafe = MarshallSolarActivityFutureEstimation(    '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\p{Digit}\p{Digit}\p{Digit}\p{Digit}F10\.(?:txt|TXT)',
#MarshallSolarActivityFutureEstimation.StrengthLevel.AVERAGE)
#DM.feed(msafe.getSupportedNames(), msafe) # Feeding the F10.7 bulletins to 
#Orekit's data manager
#atmosphere = NRLMSISE00(msafe, sun, wgs84Ellipsoid)
#isotropicDrag = IsotropicDrag(sat_list[sc_name]['cross_section'], sat_list[sc_name] 
#['cd'])
#dragForce = DragForce(atmosphere, isotropicDrag)
#propagatorBuilder.addForceModel(dragForce)

# Solar radiation pressure
#isotropicRadiationSingleCoeff = IsotropicRadiationSingleCoefficient(sat_list[sc_name]['cross_section'], sat_list[sc_name]['cr']);
#solarRadiationPressure = SolarRadiationPressure(sun, wgs84Ellipsoid.getEquatorialRadius(),
#                                            isotropicRadiationSingleCoeff)
#propagatorBuilder.addForceModel(solarRadiationPressure)


In [12]:
########## BATCH LEAST SQUARES ESTIMATOR SETUP

#Defining Optimizer
matrixDecomposer = QRDecomposer(1e-11)
optimizer = GaussNewtonOptimizer(matrixDecomposer, False)

#Defining Estimator
estimator = BatchLSEstimator(optimizer, propagatorBuilder)
estimator.setParametersConvergenceThreshold(estimator_convergence_thres)
estimator.setMaxIterations(estimator_max_iterations)
estimator.setMaxEvaluations(estimator_max_evaluations)

In [13]:
########## ADDING MEASUREMENTS

#Process Dataframe into lists for initial orbit determination
#az, el, ra, dec, datetime, slantRange = orbitalPass.getFirstAndLast()

#Add Measurements to Estimator
for j in range(0,1):#len(az)):
    year = int(datetime[j][1:5])
    month = int(datetime[j][6:8])
    day = int(datetime[j][9:11])
    hour = int(datetime[j][12:14])
    minute = int(datetime[j][15:17])
    second = int(datetime[j][18:20])
    date = AbsoluteDate(year, Month.getMonth(month), 
                        day, hour, minute, 0.0, 
                        utc)

    orekitAzEl = AngularAzEl(GroundStation(stationFrame),
					#datetime_to_absolutedate(datetime[j]),
                             date,
					JArray('double')([radians(az[j]),radians(el[j])]),
                    JArray('double')([radians(azError),radians(elError)]),
                    JArray('double')([azBaseWeight,elBaseWeight]),
                    ObservableSatellite(0))
    estimator.addMeasurement(orekitAzEl)

In [14]:
########## STATISTICAL ORBIT DETERMINATION

estimatedPropagatorArray = estimator.estimate()

JavaError: <super: <class 'JavaError'>, <JavaError object>>
    Java stacktrace:
org.orekit.errors.OrekitException: unable to compute hyperbolic eccentric anomaly from the mean anomaly after 50 iterations
	at org.orekit.errors.OrekitException.unwrap(OrekitException.java:154)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator.propagate(AbstractIntegratedPropagator.java:495)
	at org.orekit.propagation.integration.AbstractIntegratedPropagator.propagate(AbstractIntegratedPropagator.java:410)
	at org.orekit.propagation.PropagatorsParallelizer.propagate(PropagatorsParallelizer.java:141)
	at org.orekit.estimation.leastsquares.BatchLSModel.value(BatchLSModel.java:245)
	at org.hipparchus.optim.nonlinear.vector.leastsquares.LeastSquaresFactory$LocalLeastSquaresProblem.evaluate(LeastSquaresFactory.java:440)
	at org.orekit.estimation.leastsquares.BatchLSEstimator$TappedLSProblem.evaluate(BatchLSEstimator.java:616)
	at org.hipparchus.optim.nonlinear.vector.leastsquares.GaussNewtonO